## PROBLEM DESCRIPTION

The data we want to analyze is styles from a group of elements. In this case we had 4 styles so we want at least one machine for every one that it will be able to analyze the data an generate new data.

So the data that we have is a set of vectors that's the output of the app that generate grids. This grids ara a composition of elements so the outuput will be a vector of id. Each grid had 48 elements and each element can have a maximum of 255 elements in it. But is not necessary to had that number, it can be possible to have less. 

<h4 align="center">$[ x_1, x_2, x_3, ..., x_{48} ]$</h4> 
<h4 align="center">$x_1 = [id_1,id_2,_id_3, ...,id_{255}]$</h4>

Each element of the vector can have a different size from the others and the list of integers are the subcategories of style of the object that was placed in the grid. Each number is a different subcategory of style of the object placed previously.

So the first objective is to descompose the input data, a vector which element is a vector of different size, to a list of vectors that will have de same length, 48, and one element in each position. So the main goal is to find the grid wich elements has more IDs so we can set the size of total vectors, $y$ that we need to descompose the data. In case that one element of the vector has lower IDs that the max we gonna put a $0$ as the null value for our data.

<h4 align="center">$input = [[id_1^1,id_2^1,_id_3^1, ...,id_{255}^1], [id_1^2,id_2^2,_id_3^2, ...,id_{255}^2], ..., [id_1^48,id_2^48,_id_3^48, ...,id_{255}^48]]$</h4> 
<h4 align="center">$output = [[id_1^1, id_1^2,..., id_1^{48}],
[id_2^1, id_2^2,..., id_2^{48}],..., [0,0,...,id_y^{48}]$</h4> 


First of all we gonna implement a function that allows us to obtain the max value of the input data.

In [1]:
import pandas as pd
import json
import sys
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent =  currentdir + '\RBM'
sys.path.insert(0,parent)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from timeit import default_timer as timer
import numpy as np
import time
import matplotlib.pyplot as plt
import numexpr  as ne
import profile
import rbm as Rbm
import pandas
from random import randint
import random
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import BernoulliRBM
import Utils

Now we got the max length that will be the number of ouput vectors that we wanna obtain from the input. Now we need to transform the input data to the output data format. For now we gonna tet with a test vector.

In [8]:
vector_data = pd.read_json('./Data.json')
A = vector_data.values
data_modern = []
A = A.ravel()
length_vectors = len(A)
for i in range(A.shape[0]):
    len_v = len(A[i])
    tmp = []
    for j in range(len_v):
        tmp.append(A[i][j])
    tmp = np.asarray(tmp).ravel()
    data_modern.append(tmp)
data_modern = np.asarray(data_modern)
input_data = []
input_data.append(data_modern)
input_data = np.asarray(input_data)
max_elements = Utils.getMaxLengthForElements(input_data[0])
f_data = Utils.transformInputVector(input_data[0], max_elements, length_vectors)
for i in range(100):
    for k in range((length_vectors*max_elements)):
        f_data.append(f_data[k])
f_data = np.asarray(f_data)
print(f_data.shape)

(11312, 48)


At this poing we have the data formatted ready for the input for the machine. In this case we gonna use the algorithm of the library sklearn, BernoulliRBM. Also we gonna use the implementation of One Hot Vector to transform our data to One Hot Vectors.

http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.BernoulliRBM.html

http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

Now the first step to get the data in the machine is to convert all the data we formatted to one hot vectors. A one hot vector is a vector which is is a group of bits among which the legal combinations of values are only those with a single high (1) bit and all the others low (0).

In [9]:
oneHotEncoder = OneHotEncoder(262, sparse=True).fit(f_data)
oneHotData = oneHotEncoder.transform(f_data)

At this point we have 3 hot vectors which one represents the previous input data. This is a sparse matrix so is fastest to work than a simple vector and it's easy to show the results.
Then we neeed to define the parameters of the machine and fit it with de one hot vectors we had. The parameters are not specified, so we need to try whichc are the best to obtain good results

In [10]:
RBM_Machine = BernoulliRBM(n_components=262, learning_rate=0.01, n_iter=10, random_state=0, verbose=True)
RBM_Machine.fit(oneHotData)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -336.79, time = 92.12s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -168.11, time = 89.90s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -150.86, time = 89.22s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -145.36, time = 88.80s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -164.81, time = 99.21s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -156.48, time = 100.53s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -149.95, time = 96.91s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -167.42, time = 91.46s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -162.57, time = 90.88s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -152.08, time = 89.80s


BernoulliRBM(batch_size=10, learning_rate=0.01, n_components=262, n_iter=10,
       random_state=0, verbose=True)

Finally we got the output. We gonna make as much as the maximum length of samples. Then we gonna format this data and save it to a json in our folder.

In [11]:
final_output = []
for i in range(max_elements):
    x_visible = RBM_Machine.gibbs(oneHotData[random.randint(0,oneHotData.shape[0])])
    x_visible = x_visible.ravel()
    final_vector = []
    for k in range(len(x_visible)):
        if(x_visible[k] == True):
            final_vector.append(1)
        else:
            final_vector.append(0)
    final_Data = []
    for i in range(48):
        tmp = final_vector[(262*i):((i+1)*262)]
        if(1 not in tmp):
            index = 0
        else:
            index = tmp.index(1)
        final_Data.append(index)
    final_output.append(final_Data)
final_output = np.asarray(final_output)
final_data_output = []
pos = 0
for k in range(48):
    t = []
    for i in range(len(final_output)):
        t.append(final_output[i][k])
    final_data_output.append(t)
final_data_output = np.asarray(final_data_output)
output_grid = []
for i in range(48):
    v = final_data_output[i].tolist()
    v_filter = list(filter((0).__ne__,v))
    if(len(v_filter) == 0):
        v_filter.append(0)
    output_grid.append(v_filter)
print(output_grid)

[[220, 4, 216, 253, 248, 220], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [237, 121, 202, 129, 121], [12, 12, 10, 239, 10], [115, 188, 115, 3, 241, 218, 237], [177, 177, 239, 245, 218, 246], [1, 245, 8, 219], [246, 237, 121], [240, 176, 9, 135], [219, 237, 176, 219, 237, 219, 14], [198, 5], [218, 9, 218, 183, 14], [78, 218, 218, 78, 237], [153, 245, 237, 10], [10], [245], [218, 199, 13, 199], [218, 207, 219, 220], [8, 218, 218, 245, 218, 218], [253, 13, 136, 219], [246], [130, 245], [247, 219], [176, 115, 240], [245, 245, 246, 219]]


In [12]:
import codecs, json 
b = output_grid
json_file = "./output_grid.json" 
json.dump(b, codecs.open(json_file, 'w', encoding='utf-8'), sort_keys=True, indent=4)

## OTHER IMPLEMENTATION

In [13]:
oneHotEncoder = OneHotEncoder(262, sparse=False).fit(f_data)
oneHotData = oneHotEncoder.transform(f_data)
test_Data_Vector_Aux = np.array(oneHotData, dtype="float64")

In [14]:
visible_dim = oneHotData.shape[1]
hidden_dim = 500
epochs = 100
K = 1
lr = 0.1
batch_size = 10

In [15]:
rbm_ = Rbm.RBM(visible_dim=visible_dim,
               hidden_dim=hidden_dim,
               seed=42,
               mu=0, 
               sigma=0.3,
               monitor_time=True)

In [16]:
%%time
rbm_.fit(test_Data_Vector_Aux, 
         method='vectorized_CDK',
         K=K,
         lr=0.01,
         epochs=100,
         batch_size=128,
         plot_weights=False)

	Last epoch: 99 	time per epoch: 33.55	total time: 3179.17
	Training finished


Wall time: 52min 59s


In [17]:
final_output = []
for i in range(max_elements):
    x_visible, x_h = rbm_.sample_visible_from_visible(oneHotData[random.randint(0,oneHotData.shape[0])])
    x_visible = x_visible.ravel()
    final_vector = []
    for k in range(len(x_visible)):
        if(x_visible[k] == True):
            final_vector.append(1)
        else:
            final_vector.append(0)
    final_Data = []
    for i in range(48):
        tmp = final_vector[(262*i):((i+1)*262)]
        if(1 not in tmp):
            index = 0
        else:
            index = tmp.index(1)
        final_Data.append(index)
    final_output.append(final_Data)
final_output = np.asarray(final_output)
final_data_output = []
for k in range(48):
    t = []
    for i in range(len(final_output)):
        t.append(final_output[i][k])
    final_data_output.append(t)
final_data_output = np.asarray(final_data_output)
output_grid = []
for i in range(48):
    v = final_data_output[i].tolist()
    v_filter = list(filter((0).__ne__,v))
    if(len(v_filter) == 0):
        v_filter.append(0)
    output_grid.append(v_filter)
print(output_grid)

[[219], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [237], [0], [219], [219], [237, 246, 218, 219], [236], [0], [218], [0], [219], [0], [10], [219], [219], [246], [219], [14, 246], [0], [220], [245], [0], [0], [237]]


In [18]:
b = output_grid
json_file = "./output_grid_2.json" 
json.dump(b, codecs.open(json_file, 'w', encoding='utf-8'), sort_keys=True, indent=4)